In [11]:
# import dependencies
import pandas as pd
import requests
import json


In [12]:
# read csv on covid-19 covid vulnerability index data and convert to dataframe
ccvi = pd.read_csv('../resources/ccvi.csv')

# drow rows that contain any null values (there are 655 of them)
ccvi = ccvi.dropna(how='any')

# display dataframe
ccvi


,FIPS,stateName,countyName,ccvi,theme1,theme2,theme3,theme4,theme5,theme6,theme7
0,1001020100,ALABAMA,Autauga,0.441972,0.458336,0.377478,0.170811,0.841617,0.966294,0.209805,0.287591
1,1001020200,ALABAMA,Autauga,0.742619,0.562935,0.584587,0.683826,0.768947,0.982399,0.209805,0.427810
2,1001020300,ALABAMA,Autauga,0.778072,0.245094,0.645024,0.472226,0.863762,0.953086,0.724817,0.439041
3,1001020400,ALABAMA,Autauga,0.362727,0.023280,0.370119,0.192050,0.977978,0.894654,0.209805,0.425573
4,1001020500,ALABAMA,Autauga,0.816600,0.319725,0.421440,0.764060,0.753240,0.978307,0.749454,0.497597
...,...,...,...,...,...,...,...,...,...,...,...
72832,56043000200,WYOMING,Washakie,0.193098,0.632400,0.106362,0.792014,0.669875,0.126091,0.259923,0.008196
72833,56043000301,WYOMING,Washakie,0.288511,0.536803,0.358751,0.539916,0.400654,0.199599,0.259923,0.589599
72834,56043000302,WYOMING,Washakie,0.574666,0.804595,0.349800,0.613380,0.701398,0.137185,0.815509,0.273972
72835,56045951100,WYOMING,Weston,0.112481,0.476381,0.238316,0.834326,0.559188,0.149020,0.037122,0.008224


In [27]:
# dictionary for convertying state names to corresponding numbers or abbreviations
states = {
    'southcarolina': {'num': '45', 'abbr': 'SC'},
    'southdakota': {'num': '46', 'abbr': 'SD'},
    'tennessee': {'num': '47', 'abbr': 'TN'},
    'texas': {'num': '48', 'abbr': 'TX'},
    'vermont': {'num': '50', 'abbr': 'VT'},
    'utah': {'num': '49', 'abbr': 'UT'},
    'virginia': {'num': '51', 'abbr': 'VA'},
    'washington': {'num': '53', 'abbr': 'WA'},
    'westvirginia': {'num': '54', 'abbr': 'WV'},
    'wisconsin': {'num': '55', 'abbr': 'WI'},
    'wyoming': {'num': '56', 'abbr': 'WY'},
    'puertorico': {'num': '72', 'abbr': 'PR'},
    'alabama': {'num': '01', 'abbr': 'AL'},
    'alaska': {'num': '02', 'abbr': 'AK'},
    'arizona': {'num': '04', 'abbr': 'AZ'},
    'arkansas': {'num': '05', 'abbr': 'AR'},
    'california': {'num': '06', 'abbr': 'CA'},
    'colorado': {'num': '08', 'abbr': 'CO'},
    'delaware': {'num': '10', 'abbr': 'CT'},
    'districtofcolumbia': {'num': '11', 'abbr': 'DE'},
    'connecticut': {'num': '09', 'abbr': 'DC'},
    'florida': {'num': '12', 'abbr': 'FL'},
    'georgia': {'num': '13', 'abbr': 'GA'},
    'idaho': {'num': '16', 'abbr': 'ID'},
    'hawaii': {'num': '15', 'abbr': 'HI'},
    'illinois': {'num': '17', 'abbr': 'IL'},
    'indiana': {'num': '18', 'abbr': 'IN'},
    'iowa': {'num': '19', 'abbr': 'IA'},
    'kansas': {'num': '20', 'abbr': 'KS'},
    'kentucky': {'num': '21', 'abbr': 'KS'},
    'louisiana': {'num': '22', 'abbr': 'LA'},
    'maine': {'num': '23', 'abbr': 'ME'},
    'maryland': {'num': '24', 'abbr': 'MD'},
    'massachusetts': {'num': '25', 'abbr': 'MA'},
    'michigan': {'num': '26', 'abbr': 'MI'},
    'minnesota': {'num': '27', 'abbr': 'MN'},
    'mississippi': {'num': '28', 'abbr': 'MS'},
    'missouri': {'num': '29', 'abbr': 'MO'},
    'montana': {'num': '30', 'abbr': 'MT'},
    'nebraska': {'num': '31', 'abbr': 'NE'},
    'nevada': {'num': '32', 'abbr': 'NV'},
    'newhampshire': {'num': '33', 'abbr': 'NH'},
    'newjersey': {'num': '34', 'abbr': 'NJ'},
    'newmexico': {'num': '35', 'abbr': 'NM'},
    'newyork': {'num': '36', 'abbr': 'NY'},
    'northcarolina': {'num': '37', 'abbr': 'NC'},
    'northdakota': {'num': '38', 'abbr': 'ND'},
    'k6': {'num': '41', 'abbr': 'OR'},
    'pennsylvania': {'num': '42', 'abbr': 'PA'},
    'rhodeisland': {'num': '44', 'abbr': 'RI'}
}


In [34]:
# select state
state = 'Kansas'

# convert state to proper format
state = state.replace(" ", "").lower()

state_num = states[state]['num']

# all statistical categories to to be queried 
pops = 'B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E'

# create url to request data from api
url = f'https://api.census.gov/data/2019/acs/acs5?get=NAME,{pops}&for=tract:*&in=state:{state_num}'

# set returned data to a variable
response = requests.get(url).json()

# display returned data as a dataframe
pd.DataFrame(response)


,0,1,2,3,4,5,6,7,8,9,10
0,NAME,B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B03001_003E,state,county,tract
1,"Census Tract 9623, Sumner County, Kansas",2607,2528,23,0,0,0,54,20,191,962300
2,"Census Tract 9624, Sumner County, Kansas",3810,3583,14,17,0,0,214,20,191,962400
3,"Census Tract 9625, Sumner County, Kansas",4373,3870,131,118,0,0,463,20,191,962500
4,"Census Tract 9626, Sumner County, Kansas",2391,2216,0,82,27,0,192,20,191,962600
...,...,...,...,...,...,...,...,...,...,...,...
766,"Census Tract 9558, Bourbon County, Kansas",3683,3485,148,0,16,0,72,20,011,955800
767,"Census Tract 9602, Finney County, Kansas",6586,5528,93,11,435,0,3971,20,055,960200
768,"Census Tract 9603, Finney County, Kansas",2973,2512,150,30,60,0,1186,20,055,960300
769,"Census Tract 9606, Finney County, Kansas",4512,3482,20,21,69,0,3212,20,055,960600


In [35]:
# create list to store dictionaries with data for each census tract
tracts = []

# create dictionaries with population data for each census tract 
# (with properly formatted fips code)
for r in response:
    if r[0] != 'NAME':
        tracts.append({
            'FIPS': int(f'{r[8]}{r[9]}{r[10]}'),
            'total': int(r[1]),
            'white': int(r[2]),
            'black': int(r[3]),
            'native': int(r[4]),
            'asian': int(r[5]),
            'pacific': int(r[6]),
            'hispanic': int(r[7])
        })

# create dataframe with census population data
populations = pd.DataFrame(tracts)

# merge population data and ccvi data on census tract fips code
ccvi_and_pop = pd.merge(populations, ccvi, on='FIPS')

# display dataframe
ccvi_and_pop


,FIPS,total,white,black,native,asian,pacific,hispanic,stateName,countyName,ccvi,theme1,theme2,theme3,theme4,theme5,theme6,theme7
0,20191962300,2607,2528,23,0,0,0,54,KANSAS,Sumner,0.228647,0.395588,0.053051,0.570072,0.896288,0.172401,0.597183,0.020883
1,20191962400,3810,3583,14,17,0,0,214,KANSAS,Sumner,0.469417,0.390298,0.070679,0.798064,0.845337,0.192556,0.923118,0.168680
2,20191962500,4373,3870,131,118,0,0,463,KANSAS,Sumner,0.594492,0.544752,0.182355,0.912981,0.699270,0.285971,0.924527,0.204734
3,20191962600,2391,2216,0,82,27,0,192,KANSAS,Sumner,0.442346,0.237034,0.378741,0.678121,0.877410,0.188904,0.922234,0.030438
4,20205097100,2043,1946,7,3,1,0,41,KANSAS,Wilson,0.358571,0.627069,0.157010,0.627004,0.912035,0.232467,0.499655,0.025139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,20011955800,3683,3485,148,0,16,0,72,KANSAS,Bourbon,0.632923,0.224030,0.174817,0.849860,0.921358,0.646384,0.859329,0.193023
755,20055960200,6586,5528,93,11,435,0,3971,KANSAS,Finney,0.556365,0.763728,0.907861,0.625758,0.321805,0.680749,0.090554,0.251194
756,20055960300,2973,2512,150,30,60,0,1186,KANSAS,Finney,0.857719,0.718858,0.657642,0.817656,0.761766,0.476399,0.583952,0.625336
757,20055960600,4512,3482,20,21,69,0,3212,KANSAS,Finney,0.591998,0.738731,0.910470,0.817157,0.301499,0.703004,0.090554,0.185691


In [36]:
# create dictionary to hold data for each racial demographic
demogs = {
    'total': {},
    'white': {},
    'black': {},
    'native': {},
    'asian': {},
    'pacific': {},
    'hispanic': {}
}

# create list of racial groups to iterate through
races = ['total','white','black','native','asian', 'pacific', 'hispanic']

# iterate through list of races
for race in races:
    
    # calculate total population for each race
    demogs[race]['population'] = int(ccvi_and_pop[race].sum())
    
    # calculate average ccvi for each race
    demogs[race]['ccvi'] = (ccvi_and_pop[race]*ccvi_and_pop['ccvi']).sum()/demogs[race]['population']
    
    # calculate population of each race as a percentage of total population
    demogs[race]['population_percent'] = (demogs[race]['population']/demogs['total']['population'])*100

# display dictionary
demogs


{'total': {'population': 2910576,
  'ccvi': 0.33052498158231114,
  'population_percent': 100.0},
 'white': {'population': 2455905,
  'ccvi': 0.31075835726742645,
  'population_percent': 84.378659069545},
 'black': {'population': 170245,
  'ccvi': 0.49289057005410436,
  'population_percent': 5.8491858656156035},
 'native': {'population': 24010,
  'ccvi': 0.37497857109058724,
  'population_percent': 0.824922626998917},
 'asian': {'population': 85794,
  'ccvi': 0.3234368117376973,
  'population_percent': 2.9476639675445684},
 'pacific': {'population': 2382,
  'ccvi': 0.42235805406801,
  'population_percent': 0.08183947095007998},
 'hispanic': {'population': 345680,
  'ccvi': 0.5005318227367479,
  'population_percent': 11.876686951311354}}

In [37]:
# get covid data for each race by state
covid = pd.read_csv('../resources/CRDT_Data.csv')

# filter to only include data for selected state
covid = covid.loc[covid['State'] == states[state]['abbr'],:]

# filter to only include data from 2020
covid = covid.loc[covid['Date'] < 20210000,:]

# create dataframe with only relevant columns for covid cases
cases = covid[['Cases_Total','Cases_White','Cases_Black','Cases_AIAN','Cases_Asian','Cases_NHPI','Cases_Ethnicity_Hispanic']]

# create dataframe with only relevant columns for covid deaths
deaths = covid[['Deaths_Total','Deaths_White','Deaths_Black','Deaths_AIAN','Deaths_Asian','Deaths_NHPI','Deaths_Ethnicity_Hispanic']]


In [38]:
# display cases dataframe with only most recent data
cases.head(1)


,Cases_Total,Cases_White,Cases_Black,Cases_AIAN,Cases_Asian,Cases_NHPI,Cases_Ethnicity_Hispanic
1082,222433.0,152504.0,11033.0,1471.0,3931.0,NaN,31664.0


In [39]:
# display deaths dataframe with only most recent data
deaths.head(1)

,Deaths_Total,Deaths_White,Deaths_Black,Deaths_AIAN,Deaths_Asian,Deaths_NHPI,Deaths_Ethnicity_Hispanic
1082,2741.0,2110.0,149.0,21.0,34.0,NaN,205.0


In [25]:
# iterate through covid data for selected races and place data in a dictionary
for i in range(0, len(cases.columns)):
    
    # total cases for each race
    demogs[races[i]]['cases'] = int(cases[cases.columns[i]].values[0])
    
    # number of cases for each race as a percentage of total cases
    demogs[races[i]]['percent_of_cases'] = (demogs[races[i]]['cases']/demogs['total']['cases'])*100
    
    # percent discrepancy between percent of total cases and percent of total population for by each race
    # (theoretically each race should account for the same percent of cases as their percent of the population)
    demogs[races[i]]['discrepancy_percent'] = (demogs[races[i]]['percent_of_cases']/demogs[races[i]]['population_percent'])*100
    
    # total deaths for each race
    demogs[races[i]]['deaths'] = int(deaths[deaths.columns[i]].values[0])
    
    # chance of an infection resulting in death for each race
    demogs[races[i]]['chance_of_death'] = (demogs[races[i]]['deaths']/demogs[races[i]]['cases'])*100
    
    # number of deaths for each race as a percentage of total deaths
    demogs[races[i]]['percent_of_deaths'] = (demogs[races[i]]['deaths']/demogs['total']['deaths'])*100
    
    # calculated values I considered using but probably wont
#     demogs[races[i]]['discrepancy_difference'] = demogs[races[i]]['percent_of_cases'] - demogs[races[i]]['population_percent']
#     demogs[races[i]]['infected_percent'] = (demogs[races[i]]['cases']/demogs[races[i]]['population'])*100
#     demogs[races[i]]['deaths_discrepancy_percent'] = (demogs[races[i]]['percent_of_deaths']/demogs[races[i]]['population_percent'])*100
#     demogs[races[i]]['deaths_discrepancy_difference'] = demogs[races[i]]['percent_of_deaths'] - demogs[races[i]]['population_percent']
#     demogs[races[i]]['died_percent'] = (demogs[races[i]]['deaths']/demogs[races[i]]['population'])*100


# create dataframe without total population values
demographics = pd.DataFrame(demogs).drop(columns=['total'])

# display dataframe
demographics


,white,black,native,asian,pacific,hispanic
population,5.015890e+06,701334.000000,27084.000000,120654.000000,8231.000000,254791.000000
ccvi,4.377283e-01,0.645280,0.523344,0.402499,0.550582,0.553394
population_percent,8.216176e+01,11.488058,0.443644,1.976348,0.134826,4.173552
cases,1.870400e+05,34647.000000,700.000000,3266.000000,30.000000,13998.000000
percent_of_cases,4.810007e+01,8.909982,0.180015,0.839900,0.007715,3.599790
discrepancy_percent,5.854313e+01,77.558644,40.576511,42.497553,5.722135,86.252436
deaths,3.074000e+03,701.000000,9.000000,38.000000,1.000000,111.000000
chance_of_death,1.643499e+00,2.023263,1.285714,1.163503,3.333333,0.792970
percent_of_deaths,5.598252e+01,12.766345,0.163905,0.692042,0.018212,2.021490


In [26]:
# create dictionary to hold calculated values to be used in max patch
for_max = {}

# iterate through statistical categories
for row in list(demographics.index):
    
    # create a list that holds all values within the row of a statistical category
    values = demographics.loc[row].values
        
    # iterate through races
    for i in range(1, len(races)):
        
        # get population numbers
        if row == 'population':
            for_max[races[i]] = {}
            for_max[races[i]][row] = int(values[i-1])
                
        # calculate inverted ccvi values
        elif row == 'ccvi':
            for_max[races[i]]['inverted_ccvi'] = 100-(values[i-1])*100
        
        # calculate chances for where next infection will occure
        elif row == 'discrepancy_percent':
            for_max[races[i]]['chance_of_infection'] = (values[i-1]/values.sum())*100
        
        # get values for chance of infection resulting in death
        elif row == 'chance_of_death':
            for_max[races[i]][row] = values[i-1]
            
# display resulting dictionary            
for_max


{'white': {'population': 5015890,
  'ccvi': 56.227171717322896,
  'chance_of_infection': 18.815058616068672,
  'chance_of_death': 1.6434987168520103},
 'black': {'population': 701334,
  'ccvi': 35.471957364797944,
  'chance_of_infection': 24.926415385089413,
  'chance_of_death': 2.0232631973908273},
 'native': {'population': 27084,
  'ccvi': 47.6655730726296,
  'chance_of_infection': 13.040802583157884,
  'chance_of_death': 1.2857142857142856},
 'asian': {'population': 120654,
  'ccvi': 59.750050689886784,
  'chance_of_infection': 13.658202313248777,
  'chance_of_death': 1.1635027556644213},
 'pacific': {'population': 8231,
  'ccvi': 44.94177592559835,
  'chance_of_infection': 1.8390254464598028,
  'chance_of_death': 3.3333333333333335},
 'hispanic': {'population': 254791,
  'ccvi': 44.66064526193823,
  'chance_of_infection': 27.72049565597546,
  'chance_of_death': 0.7929704243463351}}